# Parallel Computation

As discussed in more detail in the introduction, parallel computation on a GPU requires multiple steps:

**1. Trigger execution on GPUs**

**2. Spawn threads**

GPUs, similar to other hardware components, are designed to be hierarchical.
To efficiently map to the underlying hardware, threads and their organization is also often hierarchical.

* CUDA/HIP organizes *thread* > *block* > *grid*
* SYCL organizes *work item* > *workgroup* > *nd-range*
* OpenMP organizes *thread* > *team* > *league*
* OpenACC organizes *thread* > *vector* > *worker* > *gang*
* Kokkos organizes *thread* > *team* > *league*

Threads are additionally grouped on the hardware level in
* *Warps* of size 32 on NVIDIA,
* *Wavefronts* of size 64 on AMD, and
* *Sub-groups*/ *sub-workgroups* on Intel.

**3. Map threads**

Each thread essentially executes the same (series of) operations.
To distinguish them, each thread has one or more IDs or indices which can be used to calculate *globally unique thread indices*.
These can then used to map threads to the parts of the work to be done.

CUDA/HIP make this explicit by provided *build-in thread variables* which, when evaluated, provide different values depending on the evaluating thread.
A global thread index is commonly computed from the block index, the block-local thread index and the block size (the number of threads per block) by evaluating
```cpp
blockIdx.x * blockDim.x + threadIdx.x
```

SYCL and Kokkos provide a global index as a single lambda parameter.

OpenMP and OpenACC internally map already existing loop indices onto threads.

Many standard algorithms don't expose indices at all, but instead work on references to elements of the input/ output data structures.

**4. Synchronization**

Waiting for the GPU to finish any outstanding work can be done either
* implicitly at the end of GPU code parts (OpenMP, OpenACC), or
* via specific API function calls.

## Implementation

Generally speaking, there are three main approaches:
* A dedicated GPU kernel (function) implemented as a separate code part and launched from the host code
* An inline kernel definition that offers better language integration but still exposes a GPU specific implementation
* Automatic conversion of code that was written to run on CPU

## Example

To demonstrate the different offloading and parallelization approaches, we consider a simple test case - increasing all elements of an array by one. \
[increase-base.cpp](../src/increase/increase-base.cpp) shows a serial CPU-only implementation.
The key part of it is the increase function.

```cpp
void increase(double* data, size_t nx) {
    for (size_t i0 = 0; i0 < nx; ++i0) {
        data[i0] += 1;
    }
}
```

Other things our application does are:
* parse command line arguments
    * `nx`, the number of elements in the vector to be processed
    * `nItWarmUp`, the number of warm-up iterations
    * `nIt`, the number of timed iterations
* allocate an array with `nx` elements
* initialize the array such that each element holds a value equal to its index 
* call `increase` `nItWarmUp` times
* call `increase` `nIt` times and take the time required for it
* print statistics and estimated performance metrics
* check that all array elements have the expected value
* deallocate the array

The code can be compiled and executed with the following cells

In [ ]:
!g++ -O3 -march=native -std=c++17 -o ../build/increase/increase-base ../src/increase/increase-base.cpp

In [ ]:
!../build/increase/increase-base

In [ ]:
!g++ -O3 -march=native -std=c++17 -fopenmp -o ../build/increase/increase-omp-host ../src/increase/increase-omp-host.cpp

In [ ]:
!../build/increase/increase-omp-host

## OpenMP

**1.** OpenMP allows code execution on GPUs by introducing *target regions*.

```cpp
#pragma omp target
for (size_t i0 = 0; i0 < nx; ++i0) {
    data[i0] += 1;
}
```

**2.** This code executes the loop on the GPU *serially*.
Parallelism can be created with `teams` and `parallel`.

**3.** Mapping loop iterations to the spawned threads can be done with `distribute` and `for`.
Without further specification, the number of teams and threads per team are chosen by the compiler.

```cpp
#pragma omp target teams distribute parallel for
for (size_t i0 = 0; i0 < nx; ++i0) {
    data[i0] += 1;
}
```

**4.** Synchronization is implicit at the end of the target region.

The full example code is available at [increase-omp-target-expl.cpp](../src/increase/increase-omp-target-expl.cpp) and [increase-omp-target-mm.cpp](../src/increase/increase-omp-target-mm.cpp).
They can be build and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -target=gpu -o ../build/increase/increase-omp-target-expl ../src/increase/increase-omp-target-expl.cpp

In [ ]:
!../build/increase/increase-omp-target-expl

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -target=gpu -gpu=mem:managed -o ../build/increase/increase-omp-target-mm ../src/increase/increase-omp-target-mm.cpp

In [ ]:
!../build/increase/increase-omp-target-mm

## OpenACC

OpenACC allows a similar approach to the previously discussed OpenMP target offloading by using `parallel` (thread spawning) and `loop` (work distribution).\
Whether execution is ultimately done on CPU or GPU is specified by providing different arguments when compiling.\
Without further specification, the number of gangs and workers, and vector size are chosen by the compiler.

```cpp
#pragma acc parallel loop
for (size_t i0 = 0; i0 < nx; ++i0) {
    data[i0] += 1;
}   // implicit synchronization
```

As with OpenMP, this code will trigger the compiler to apply parallelization - whether it is legal or not (e.g. in case of race conditions due to inter-iteration data dependencies).
Alternatively, `kernels` may be used to give more control to the compiler.
It will then
* perform a dependency analysis and only parallelize loops if no loop dependencies were found, and
* perform loop/ kernel transformations including fusion.

```cpp
#pragma acc kernels
{
    for (size_t i0 = 0; i0 < nx; ++i0) {
        data[i0] += 1;
    }
    /* potentially more work */
}
```

The full example code is available at [increase-openacc-expl.cpp](../src/increase/increase-openacc-expl.cpp) and [increase-openacc-mm.cpp](../src/increase/increase-openacc-mm.cpp).
They can be build and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -acc=gpu -target=gpu -o ../build/increase/increase-openacc-expl ../src/increase/increase-openacc-expl.cpp

In [ ]:
!../build/increase/increase-openacc-expl

In [ ]:
!nvc++ -O3 -std=c++17 -acc=gpu -target=gpu -gpu=mem:managed -o ../build/increase/increase-openacc-mm ../src/increase/increase-openacc-mm.cpp

In [ ]:
!../build/increase/increase-openacc-mm

## Modern C++

An alternative to loop-based operations and their parallelization is using STL algorithms.\
They can be marked for parallelization by providing an additional *execution policy*, and GPU offloading can be triggered via compiler arguments.

```cpp
std::transform(std::execution::par_unseq, data, data + nx, data,
               [=](auto data_item) {
                   return data_item + 1;
               }); // implicit synchronization
```

If indices are required (e.g. for accessing information in some sort of neighborhood), there are two main alternatives.

Reconstruction of the index via pointer arithmetic ...
```cpp
std::for_each(std::execution::par_unseq, data, data + nx,
              [=](const auto& data_item) {
                  const size_t i0 = &data_item - data;
                  data[i0] += 1;
              });
```

... or using a thrust `counting_iterator` (also available on AMD through *rocThrust*)
```cpp
std::for_each(std::execution::par_unseq, thrust::make_counting_iterator<size_t>(0), thrust::make_counting_iterator<size_t>(nx),
              [=](const auto &i0) {
                  data[i0] += 1;
              });
```

The full example code is available at [increase-std-par.cpp](../src/increase/increase-std-par.cpp).
It can be build and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -stdpar=gpu -target=gpu -gpu=cc86 -o ../build/increase/increase-std-par ../src/increase/increase-std-par.cpp

In [ ]:
!../build/increase/increase-std-par

## Thrust

For more control and support for additional computational patterns, thrust can be a viable alternative.
It provides GPU-accelerated versions of many STL algorithms as well as additional ones.
They also receive an *execution policy* as argument, but in this case it prescribes where to execute the computation.

```cpp
thrust::transform(thrust::device, data.begin(), data.end(), data.begin(),
                  [=] __host__ __device__ (double data_elem) {
                      return data_elem + 1;
                  }); // implicit synchronization
```

As before, using a counting iterator is also possible.

```cpp
double *data_ptr = thrust::raw_pointer_cast(data.data());
thrust::for_each(thrust::device, thrust::make_counting_iterator<size_t>(0), thrust::make_counting_iterator<size_t>(nx),
                 [=] __host__ __device__ (size_t i0) {
                     data_ptr[i0] += 1;
                 });
```

Alternatively, thrust also provides the `tabulate` pattern which applies a transform on the *index* of each element.

```cpp
double *data_ptr = thrust::raw_pointer_cast(data.data());
thrust::tabulate(thrust::device, data.begin(), data.end(),
                 [=] __host__ __device__ (size_t i0) {
                     return data_ptr[i0] + 1;
                 });
```

The full example code is available at [increase-thrust-expl.cu](../src/increase/increase-thrust-expl.cu) and [increase-thrust-mm.cu](../src/increase/increase-thrust-mm.cu).
They can be build and executed using the following cells.

In [ ]:
!nvcc -O3 -std=c++17 --extended-lambda -arch=sm_86 -o ../build/increase/increase-thrust-expl ../src/increase/increase-thrust-expl.cu

In [ ]:
!../build/increase/increase-thrust-expl 

In [ ]:
!nvcc -O3 -std=c++17 --extended-lambda -arch=sm_86 -o ../build/increase/increase-thrust-mm ../src/increase/increase-thrust-mm.cu

In [ ]:
!../build/increase/increase-thrust-mm 

## Kokkos

Kokkos provides its own abstraction for loops executed in parallel.
Depending on how kokkos was compiled, this will map to either CPU or GPU execution spaces.

```cpp
Kokkos::parallel_for(
    Kokkos::RangePolicy<>(0, nx),
        KOKKOS_LAMBDA(const size_t i0) {
            data(i0) += 1;
        });
```

Tuning of the thread hierarchy composition can be done by specifying an additional *team policy*.

Synchronization for GPUs is not implicit but can be done by calling
```cpp
Kokkos::fence();
```

The full example code is available at [increase-kokkos.cpp](../src/increase/increase-kokkos.cpp).
It can be build and executed using the following cells.

In [ ]:
!g++ -O3 -march=native -std=c++17 -I/root/kokkos/install-serial/include -L/root/kokkos/install-serial/lib -o ../build/increase/increase-kokkos-serial ../src/increase/increase-kokkos.cpp -lkokkoscore -ldl

In [ ]:
!../build/increase/increase-kokkos-serial

In [ ]:
!/root/kokkos/install-cuda/bin/nvcc_wrapper -O3 -march=native -std=c++17 -arch=sm_86 --expt-extended-lambda --expt-relaxed-constexpr -I/root/kokkos/install-cuda/include -L/root/kokkos/install-cuda/lib -o ../build/increase/increase-kokkos-cuda ../src/increase/increase-kokkos.cpp -lkokkoscore -ldl -lcuda

In [ ]:
!../build/increase/increase-kokkos-cuda

## SYCL

SYCL also provides its own abstraction for parallel loops, but additionally requires a *handler* and a *queue*.
The following example assumes an already initialized `sycl::queue` named `q`.

```cpp
q.submit([&](sycl::handler &h) {
    h.parallel_for(nx, [=](auto i0) {
        data[i0] += 1;
    });
});
```

Tuning the workgroup size is possible as well by providing global and local sizes, i.e. the total number of threads and the number of threads per workgroup.
Note, that these two need to be evenly divisible and that any additionally spawned threads may need to be masked.

```cpp
auto local_size = 256;
auto global_size = ceilingDivide(nx, local_size) * local_size;
q.submit([&](sycl::handler &h) {
    h.parallel_for( { global_size, local_size }
        nx, [=](auto i0) {
        if (i0 < nx) {
            data[i0] += 1;
        }
    });
});
```

In any case, synchronization is done by calling
```cpp
q.wait();
```

When using buffers, additional accessors must be created for accessing data

```cpp
q.submit([&](sycl::handler &h) {
    auto data = b_data.get_access(h, sycl::read_write);

    h.parallel_for(nx, [=](auto i0) {
        dest[i0] = src[i0] + 1;
    });
});
```

The full example code is available at [increase-sycl-expl.cpp](../src/increase/increase-sycl-expl.cpp), [increase-sycl-mm.cpp](../src/increase/increase-sycl-mm.cpp) and [increase-sycl-buffer.cpp](../src/increase/increase-sycl-buffer.cpp).
They can be build and executed using the following cells.

In [ ]:
!icpx -O3 -march=native -std=c++17 -fsycl -fsycl-targets=nvptx64-nvidia-cuda -Xsycl-target-backend --cuda-gpu-arch=sm_86 -o ../build/increase/increase-sycl-buffer ../src/increase/increase-sycl-buffer.cpp

In [ ]:
!../build/increase/increase-sycl-buffer

In [ ]:
!icpx -O3 -march=native -std=c++17 -fsycl -fsycl-targets=nvptx64-nvidia-cuda -Xsycl-target-backend --cuda-gpu-arch=sm_86 -o ../build/increase/increase-sycl-expl ../src/increase/increase-sycl-expl.cpp

In [ ]:
!../build/increase/increase-sycl-expl

In [ ]:
!icpx -O3 -march=native -std=c++17 -fsycl -fsycl-targets=nvptx64-nvidia-cuda -Xsycl-target-backend --cuda-gpu-arch=sm_86 -o ../build/increase/increase-sycl-mm ../src/increase/increase-sycl-mm.cpp

In [ ]:
!../build/increase/increase-sycl-mm

## CUDA/ HIP

CUDA/HIP utilize separate *kernel* functions which are *launched* from the host code.
By convention, they must return `void` and are marked with the `__global__` keyword.

```cpp
__global__ void increase(double* data, size_t nx) {
    for (size_t i0 = 0; i0 < nx; ++i0) {
        data[i0] += 1;
    }
}
```

The kernel can be configured by providing an *execution configuration* in triple-chevron syntax.

```cpp
increase<<<1, 1>>>(d_data, nx);
```

The above example runs on GPU, but all work is done in a single thread.
Additional parallelization has to be done manually.
One straight-forward way is assigning the work of a single loop iteration to a single thread, and to spawn as many threads as there are loop iterations.
The mapping is done by having each thread evaluate the build-in thread variables to compute a unique global index or unique data index.

```cpp
__global__ void increase(double* data, size_t nx) {
    const size_t i0 = blockIdx.x * blockDim.x + threadIdx.x;
    dest[i0] = src[i0] + 1;
}
```

```cpp
auto numThreadsPerBlock = 256;
auto numBlocks = nx / numThreadsPerBlock;
increase<<<numBlocks, numThreadsPerBlock>>>(d_data, nx);
```

While the above example works if nx is evenly divisible by the block size, it will not in all other cases.
The most prominent way to fix this is to spawn an additional block and not have all threads of that block doing computations.

```cpp
__global__ void increase(double* data, size_t nx) {
    const size_t i0 = blockIdx.x * blockDim.x + threadIdx.x;

    if (i0 < nx)
        dest[i0] = src[i0] + 1;
}
```

```cpp
auto numThreadsPerBlock = 256;
auto numBlocks = ceilingDivide(nx, numThreadsPerBlock);
increase<<<numBlocks, numThreadsPerBlock>>>(d_data, nx);
```

The full example code is available at [increase-cuda-expl.cpp](../src/increase/increase-cuda-expl.cpp) and [increase-cuda-mm.cpp](../src/increase/increase-cuda-mm.cpp).
They can be build and executed using the following cells.

In [ ]:
!nvcc -O3 -std=c++17 -arch=sm_86 -o ../build/increase/increase-cuda-expl ../src/increase/increase-cuda-expl.cu

In [ ]:
!../build/increase/increase-cuda-expl

In [ ]:
!nvcc -O3 -std=c++17 -arch=sm_86 -o ../build/increase/increase-cuda-mm ../src/increase/increase-cuda-mm.cu

In [ ]:
!../build/increase/increase-cuda-mm

## Next Step

Head to the [next steps](./next-steps.ipynb) notebook.